In [1]:
import os
import sys

if 'has_changed_dir' not in globals():
    repo_path = os.path.abspath(os.path.join('..'))
    
    if repo_path not in sys.path:
        sys.path.append(repo_path)
    
    os.chdir(repo_path)
    
    globals()['has_changed_dir'] = True

import numpy as np
from tqdm import tqdm

import neunet as nnet
import neunet.nn as nn
from data_loader import load_mnist
from neunet.optim import Adam


In [2]:
image_size = (1, 28, 28)

training_dataset, test_dataset, training_targets, test_targets = load_mnist()
training_dataset = (
    training_dataset / 127.5 - 1
)  # normalization: / 255 => [0; 1]  #/ 127.5-1 => [-1; 1]
test_dataset = test_dataset / 127.5 - 1  # normalization: / 255 => [0; 1]  #/ 127.5-1 => [-1; 1]

device = "cpu"


In [3]:
class RecurrentClassifier(nn.Module):
    def __init__(self):
        super(RecurrentClassifier, self).__init__()

        self.lstm1 = nn.LSTM(28, 128, return_sequences=True)
        self.lstm2 = nn.LSTM(128, 128, return_sequences=False)
        self.fc1 = nn.Linear(128, 10)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.sigmoid(x)

        return x


classifier = RecurrentClassifier().to(device)
loss_fn = nn.MSELoss()
optimizer = Adam(classifier.parameters(), lr=0.0001)

In [4]:
def one_hot_encode(labels):
    one_hot_labels = np.zeros((labels.shape[0], 10))

    for i in range(labels.shape[0]):
        one_hot_labels[i, int(labels[i])] = 1

    return one_hot_labels



In [5]:

batch_size = 100
epochs = 5

for epoch in range(epochs):
    tqdm_range = tqdm(range(0, len(training_dataset), batch_size), desc="epoch " + str(epoch))
    for i in tqdm_range:
        batch = training_dataset[i : i + batch_size]

        batch = batch.reshape(batch.shape[0], image_size[1], image_size[2])

        batch = nnet.tensor(batch, device=device)

        labels = nnet.tensor(one_hot_encode(training_targets[i : i + batch_size]), device=device)

        optimizer.zero_grad()

        outputs = classifier(batch)

        loss = loss_fn(outputs, labels)

        loss.backward()

        optimizer.step()

        tqdm_range.set_description(f"epoch: {epoch + 1}/{epochs}, loss: {loss.item():.7f}")

epoch: 5/5, loss: 0.0152862: 100%|██████████| 600/600 [03:20<00:00,  2.99it/s]


In [6]:
# evaluate
correct = 0
total = 0


for i in tqdm(range(len(test_dataset)), desc="evaluating"):
    img = test_dataset[i]
    img = img.reshape(1, image_size[1], image_size[2])
    img = nnet.tensor(img, device=device)

    outputs = classifier(img)
    predicted = np.argmax(outputs.detach().cpu().numpy())

    total += 1
    correct += predicted == test_targets[i]

print("Accuracy of the network on the 10000 test images: %d %%" % (100 * correct / total))


evaluating: 100%|██████████| 10000/10000 [04:16<00:00, 38.95it/s]

Accuracy of the network on the 10000 test images: 91 %
